# 9. Closed-book QA with T5

## 강의소개

- 9강과 10강에서는 retriever-reader 방법론 외의 ODQA를 푸는 다양한 방법들에 대해 알아보겠습니다.
- 9강에서는 최근 등장한 T5 모델을 활용한 closed-book QA에 대해 배워보겠습니다.
- T5의 핵심은 text-to-text format을 활용한다는 것인데, text-to-text format이 무엇인지, 그리고 이것을 QA에는 어떻게 활용할 수 있을지에 대해 이번 강의에서 중점적으로 다뤄보겠습니다.

<br>

## Further Reading

- [Exploring the limits of transfer learning with a unified text-to-text transformer(T5)](https://arxiv.org/abs/1910.10683)
- [How much knowledge can you pack into the parameters of language model?](https://arxiv.org/abs/2002.08910)
- [UNIFIEDQA: Crossing Format Boundaries with a Single QA System](https://arxiv.org/abs/2005.00700)

<br>

## Reference

- Raffel et al., 2020, Exploring the Limits of Transfer Learning with a Unified Text-to-Text Transformer
- Khashabi et al., 2020, UNIFIEDQA: Crossing Format Boundaries with a Single QA System
- Roberts et al., 2020, How Much Knowledge Can You Pack Into the Parameters of a Language Model?
- Petroni et al., 2019. Language Models as Knowledge Bases? 
- Radford et al., 2019. Language Models are Unsupervised Multitask Learners

<br>

## 9.1 Closed-book Question Answering

- Idea of Closed-book Question Answering
- Zero-shot QA performance of GPT-2
- Open-book QA vs Closed-book QA

<br>

### 9.1.1 Current approaches of building QA system

- ODQA 시스템을 접근하는 방식은 문서를 주거나, 문서가 주어지지 않았을 때는 전체 웹 코퍼스에서 해당되는 문서를 찾아서 모델에게 전달하고 정답을 찾는 형태였다.
- 2018, 2019년까지 대부분 이러한 접근 방법을 택했다.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1soq0_Jm4Qd4TAuceVSgJGJSmZX7IesZF' width=800/>

<br>

### 9.1.2 Idea of Closed-book Question Answering

- 모델이 이미 사전학습으로 대량의 지식을 학습했다면, 사전학습 언어모델 자체가 이미 하나의 knowledge storage 라고 볼 수 있지 않을까?
- 즉, 굳이 다른 곳에서 지식을 가져와야할 필요가 없지 않을까?
- 문서를 따로 입력으로 받지 않고 모델 안에 모델 안에 모든 정보가 포함되어 있어야 한다.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1sucHieD8psjnRhny0DIOCoJ4ydZStB2t' width=800/>

<br>

### 9.1.3 Zero-shot QA performance of GPT-2

- GPT-2를 통해 Closed-book QA가 처음 선보여졌다.
- 사전학습 시 전혀 본 적없는 Natural Questions 데이터셋에도 어느 정도 대답이 가능함

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1t1EKZrkallLkaOI2f6iwQVd1WG3Z9Tnm' width=800/>

<br>

### 9.1.4 Open-book QA vs. Closed-book QA

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1t6irL99JZxgIKEc4NAg1ru7MsRi3piAK' width=800/>

<br>

## 9.2 Text-to-Text Format

- Closed-book QA as Text-to-Text Format
- Text-to-Text Format
- Pre-training T5
- Using T5 for Closed-book QA

<br>

### 9.2.1 Closed-book QA as Text-to-Text Format

- Closed-book QA에 사용되는 방법은 Generation-based MRC와 유사함 (3강 참고)
- 단, 입력에 지문(Context)이 없이 질문만 들어간다는 것이 차이점
- 사전 학습된 언어 모델은 BART와 같은 seq-to-seq 형태의 Transformer 모델을 사용함
- Text-to-Text format에서는 각 입력값(질문)과 출력값(답변)에 대한 설명을 맨 앞에 추가함

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1tAJLyd75RXry7kKNHUis9Z9_wvhOtzzw' width=600/>

<br>

### 9.2.2 Text-to-Text Format

- Text-to-text problem
  - input으로 text를 받아서, output으로 새로운 text를 생성하는 문제
- 다양한 text processing problem을 Text-to-text 문제로 변형할 수 있다.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1tPSLyz1aj7MbVQr6AOFwhwDWW2d2U5Nm' width=800/>

<br>

### 9.2.3 Text-to-Text Format Example 1

- 특정 task에 알맞은 output text를 생성하도록 <mark>Task-specific prefix를 추가</mark>

<br>

Ex1) Machine translation

- prefix = `translate A to B`
- `A`: source language
- `B`: target language

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1tZ_GjiaqHiLS6hCDd2sl-Ry77QnVc1Xy' width=800/>

<br>

### 9.2.4 Text-to-Text Format Example 2

Ex2) Text Classification (MNLI)

- MNLI
  - 두 개의 sentence (premise, hypothesis)가 주어지고, 이 둘의 관계를 예측하는 task
  - 관계의 종류: neutral, contradiction, entailment
- Input
  - `mnli hypothesis: <sent 1> premise: <sent 2>`
- Output
  - `neutral` or `contradiction` or `entailment`

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1t_HKHUXX4mdpEp7-udW94Ab5CeLev3xG' width=800/>

<br>

### 9.2.5 Model Overview

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1t_omQR6wpomlhb17lRBNHNcLPEv_LLNh' width=800/>

<br>

### 9.2.6 T5

- Text-to-Text format 이라는 형태로 데이터의 입출력을 만들어 거의 모든 자연어처리 문제를 해결하도록 학습된 seq-to-seq 형태의 Transformer 모델

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1tdI7qSztVDL7egVWg9G6kj8CKPR_QeVn' width=800/>

<br>

### 9.2.7 Pre-training T5

- 다양한 모델 구조, 사전학습 목표, 사전학습용 데이터, Fine-tuning 방법 등을 체계적으로 실험함
- 가장 성능이 좋은 방식들을 선택하여 방대한 규모의 모델을 학습시킴
- C4라는 대용량의 학습 데이터를 사용

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1tfCmUbCg2NawAi2lPiNOCVYWpAxI9_R3' width=800/>

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1tgEXstaJ-kagdmJRpfr9RaVJhYgJY7ri' width=600/>

- 가장 큰 T5 모델인 T5-xlarge의 경우 파라미터의 갯수가 11B에 육박한다.

<br>

### 9.2.8 Using T5 for Closed-book QA

- Fine-tuning T5
  - 미리 학습된 pre-trained T5를 활용
- Fine-tuning
  - MRC 데이터셋 (TriviaQA, WebQuestions, Natural Questions)의 QA pair를 활용

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1tgYg2KsC79jABszQ2PcUTw5qZMtLqHFL' width=800/>

- MRC 데이터셋에 제공되는 supporting document는 무시
- Input
  - Task-specific prefix 추가
  - `trivia question: <question>`
- Natural Questions와 같이 답이 여러 개인 경우
  - `answer: <answer 1> answer: <answer 2>`

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1th1DCb9pxCYoFmLJwrwIsOLju2W7GPib' width=950/>

<br>

## 9.3 Experiment Results & Analysis

- Experiment Setting
- Quantitative Examples
- Prediction Examples
- False negatives
- Limitation

<br>

### 9.3.1 Experiment Setting

- **Dataset**
  - Open-domain QA 데이터셋 또는 MRC 데이터셋에서 지문을 제거하고 질문과 답변만 남긴 데이터셋을 활용
- **Salient(핵심적인) Span Masking**
  - 고유 명사, 날짜 등 의미를 갖는 단위에 속하는 토큰 범위를 마스킹한 뒤 학습
    - BERT에서 랜덤으로 masking 하다 보니 조사같은 토큰이 masking되는 경우가 발생하여 효율이 떨어졌다.
  - Pre-trained 체크포인트에서 추가로 pre-training 함
- **Fine-tuning**
  - Pre-trained T5 체크포인트를 Open-domain QA 학습 데이터셋으로 추가 학습

<br>

### 9.3.2 Experiment Examples

- T5를 이용한 Closed-book Question Answering 예시
- task prefix: `trivia question:` (사소한 질문)

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1thBauyxTkBRultDRWH94aAxhFgrGG5LE' width=800/>

<br>

### 9.3.3 Quantitative Examples

- 문서 검색 후 기계 독해를 하는 대부분의 Open-book 스타일 모델보다 뛰어난 성능을 보여줌
- 모델 크기가 커질수록 성능이 증가함
- Salient Span Masking이 성능을 크게 끌어올림

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1tj7Yzuajkt2v5r-bj_4mlxl3B3OwZ2HD' width=400/>

<br>

### 9.3.4 Prediction Examples

- T5를 이용한 Closed-book Question Answering 예시

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1tqgFj2yg31ieV-rdjELJRW-U07cQZtGj' width=800/>

<br>

### 9.3.5 False negatives

- T5 모델의 오답 분석
- Exact match 기준으로 오답으로 채점된 결과를 사람이 평가한 결과 오답이 아닌 경우

<br>

1. Phrasing Mismatch
  - 정답에 대한 표현이 다른 경우
  - 정답이긴 하지만 표현이 다른 경우
2. Incomplete Annotation
  - 정답이 여러 개일 수 있으나 하나만 정답으로 처리되는 경우
3. Unasnwerable
  - 질문을 한 시간이나 문맥에 따라서 정답이 달라지는 경우
  - ex) 미국의 대통령

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1tsY5Ha-LfhGfAFZmh95DxAdzGTJ0jsGt' width=800/>

- 62%는 실제로 틀린 답(True Negative)이였지만 나머지는 실제로는 올바른 답이였다.

<br>

### 9.3.6 Limitations of Closed-book QA

- Closed-book QA의 한계점 및 앞으로의 개선 방향

<br>

1. 모델의 크기가 커서 계산량이 많고 속도가 느림
  - 더 효율적인 모델 필요
2. 모델이 어떤 데이터로 답을 내는 지 알 수 없음
  - 결과의 해석 가능성(interpretability)을 높이는 연구 필요
3. 모델이 참조하는 지식을 추가하거나 제거하기 어려움